In [ ]:
from seisbench.data import WaveformDataset
from seisbench.data import MultiWaveformDataset
import sys
import os
import glob

In [ ]:
lib_path = [r'C:\Users\ikahbasi\OneDrive\Applications\GitHub\SeisRoutine',
            r'C:\Users\ikahb\OneDrive\Applications\GitHub\SeisRoutine']
for path in lib_path:
    sys.path.append(path)
##########################################################################
import SeisRoutine.catalog as src
import SeisRoutine.waveform as srw
import SeisRoutine.config as srconf
import SeisRoutine.statistics as srs

In [ ]:
init_cfg = srconf.load_config('0-init-cfg.yml')

lst_datasets = []
for cfg_path in glob.glob(os.path.join(init_cfg.target_config_filepath, '*-last_run-cfg.yml')):
    print(cfg_path)
    cfg = srconf.load_config(cfg_path)
    dataset = WaveformDataset(
        path=cfg.mk_dataset.path.outputs.dataset,
        component_order=cfg.mk_dataset.parameters.component_order,
        # dimension_order=cfg.mk_dataset.parameters.dimension_order,
        sampling_rate=cfg.mk_dataset.parameters.sampling_rate
        )
    lst_datasets.append(dataset)

In [ ]:
combined_dataset = MultiWaveformDataset(lst_datasets)

In [ ]:
list(combined_dataset.metadata.keys())

In [ ]:
from seisbench.data import WaveformDataWriter
from pathlib import Path
from datetime import datetime

In [ ]:
today_str = datetime.today().strftime('%Y-%m-%dT%H-%M-%S')

base_path = Path(r'C:\Users\ikahbasi\Documents\Merged_All_DataSets_2025-07-10 (Ahar-Ilam-Kaki-Qeshm)')
metadata_path = base_path / "metadata.csv"
waveforms_path = base_path / "waveforms.hdf5"

In [ ]:
with WaveformDataWriter(metadata_path=metadata_path,
                        waveforms_path=waveforms_path) as writer:
    writer.data_format = {
        "dimension_order":     cfg.mk_dataset.parameters.dimension_order,
        "component_order":     cfg.mk_dataset.parameters.component_order,
        "sampling_rate":       cfg.mk_dataset.parameters.sampling_rate,
        "measurement":         cfg.mk_dataset.parameters.measurement,
        "unit":                cfg.mk_dataset.parameters.unit,
        "instrument_response": cfg.mk_dataset.parameters.instrument_response,
    }
    for idx in range(len(combined_dataset)):
        waveform, metadata = combined_dataset.get_sample(idx)
        del metadata['trace_name']
        del metadata['index']
        writer.add_trace(metadata=metadata, waveform=waveform)